In [1]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models, regularizers, optimizers
from tensorflow.keras.utils import to_categorical

In [2]:
GENE = "001000100000011101000000111011101001010000011111001110000101000100100"
INPUT_SHAPE = 28

LEARNING_RATE_DICT = {
    0: 1 * 10 ** (-5), 1: 5 * 10 ** (-5),
    2: 1 * 10 ** (-4), 3: 5 * 10 ** (-4),
    4: 1 * 10 ** (-3), 5: 5 * 10 ** (-3),
    6: 1 * 10 ** (-2), 7: 5 * 10 ** (-2),
}

DENSE_TYPE_DICT = {
    0: "recurrent", 1: "LSTM", 2: "GRU", 3: "feed-forward"
}

REGULARIZATION_DICT = {
    0: "l1", 1: "l2", 2: "l1l2", 3: None
}

ACTIVATION_DICT = {
    0: "relu",
    1: "linear"
}

RGL_DCT = {
            0: regularizers.l1(1e-4),
            1: regularizers.l2(1e-4),
            2: regularizers.l1_l2(l1=1e-4, l2=1e-4),
            3: None
        }

In [3]:
def binary_to_decimal(bits):
    return int("".join(map(str, bits)), 2)

def get_batch_size(gene):
        return [25, 50, 100, 15][binary_to_decimal(gene[:2])]

def get_convol_layers_num(gene):
    return 1 + binary_to_decimal(gene[2:4])

def get_kernels_num(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[4 + i * 10: 4 + i * 10 + 3]
        result.append(2 ** (binary_to_decimal(binary) + 1))
    return result

def get_kernel_sizes(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[7 + i * 10: 7 + i * 10 + 3]
        result.append(2 + binary_to_decimal(binary))
    return result

def get_pooling(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[10 + i * 10: 10 + i * 10 + 3]
        result.append(1 + binary_to_decimal(binary))
    return result

def get_convol_activation(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[13 + i * 10: 13 + i * 10 + 1]
        result.append(ACTIVATION_DICT[binary_to_decimal(binary)])
    return result

def get_dense_layers_num(gene):
    return 1 + binary_to_decimal([gene[44]])

def get_dense_type(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[45 + i * 9: 45 + i * 9 + 2]
        result.append(DENSE_TYPE_DICT[binary_to_decimal(binary)])
    return result

def get_neurons_num(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[47 + i * 9: 47 + i * 9 + 3]
        result.append(2 ** (binary_to_decimal(binary) + 3))
    return result

def get_dense_activation(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[50 + i * 9: 50 + i * 9 + 1]
        result.append(ACTIVATION_DICT[binary_to_decimal(binary)])
    return result

def get_regularization(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[51 + i * 9: 51 + i * 9 + 2]
        result.append(binary_to_decimal(binary))
    return result

def get_dropout(gene, layers_num):
    result = []
    for i in range(layers_num):
        binary = gene[53 + i * 9: 53 + i * 9 + 1]
        result.append(binary_to_decimal(binary) / 2)
    return result

def get_optimizer(gene):
    binary = gene[63: 66]
    return binary_to_decimal(binary)

def get_learning_rate(gene):
    binary = gene[66: 69]
    return LEARNING_RATE_DICT[binary_to_decimal(binary)]

def get_components(gene):
    dct = {}

    dct["b"] = get_batch_size(gene)

    # Convolutional layers
    dct["nc"] = get_convol_layers_num(gene)
    dct["ck"] = get_kernels_num(gene, dct["nc"])
    dct["cs"] = get_kernel_sizes(gene, dct["nc"])
    dct["cp"] = get_pooling(gene, dct["nc"])
    dct["ca"] = get_convol_activation(gene, dct["nc"])

    # Dense layers
    dct["nd"] = get_dense_layers_num(gene)
    dct["dt"] = get_dense_type(gene, dct["nd"])
    dct["dn"] = get_neurons_num(gene, dct["nd"])
    dct["da"] = get_dense_activation(gene, dct["nd"])
    dct["dd"] = get_dropout(gene, dct["nd"])
    dct["dr"] = get_regularization(gene, dct["nd"])

    # Learning parameters
    dct["n"] = get_learning_rate(gene)
    dct["f"] = get_optimizer(gene)

    return dct

In [4]:
def add_dense_layers(components, model):
    new_model = model
    if components["nd"] == 1:

        if components["dt"][0] == "feed-forward":
            new_model.add(layers.Flatten())
            new_model.add(
                layers.Dense(components["dn"][0], 
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0]
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "recurrent":
            last_output = new_model.layers[-1].output_shape
            new_model.add(
                tf.keras.layers.Reshape((last_output[1] * last_output[2], last_output[3]), 
                input_shape=last_output
            ))
            new_model.add(layers.SimpleRNN(
                components["dn"][0], 
                kernel_regularizer=RGL_DCT[components["dr"][0]],
                activation=components["da"][0]
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "GRU":
            last_output = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape((
                last_output[1] * last_output[2], last_output[3]), 
                input_shape=last_output
            ))
            new_model.add(layers.GRU(
                components["dn"][0], 
                kernel_regularizer=RGL_DCT[components["dr"][0]],
                activation=components["da"][0]
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "LSTM":
            new_model.add(layers.TimeDistributed(layers.LSTM(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0]
            )))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.Flatten())
    else:

        if components["dt"][0] == "feed-forward" and components["dt"][1] == "feed-forward":
            new_model.add(layers.Flatten())
            new_model.add(layers.Dense(
                components["dn"][0], 
                kernel_regularizer=RGL_DCT[components["dr"][0]],
                activation=components["da"][0]))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.Dense(
                components["dn"][1], 
                kernel_regularizer=RGL_DCT[components["dr"][1]],
                activation=components["da"][1]))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "feed-forward" and components["dt"][1] == "recurrent":
            new_model.add(layers.Flatten())
            new_model.add(layers.Dense(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0]
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape((
                last_shape[1] // 2, 2), 
                input_shape=last_shape
            ))
            new_model.add(layers.SimpleRNN(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]],
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "feed-forward" and components["dt"][1] == "GRU":
            new_model.add(layers.Dense(
                components["dn"][0], 
                kernel_regularizer=RGL_DCT[components["dr"][0]],
                activation=components["da"][0]
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape((
                last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.GRU(
                components["dn"][1], 
                kernel_regularizer=RGL_DCT[components["dr"][1]],
                activation=components["da"][0]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "feed-forward" and components["dt"][1] == "LSTM":
            new_model.add(layers.Dense(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0]
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.TimeDistributed(layers.LSTM(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            )))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.Flatten())

        if components["dt"][0] == "recurrent" and components["dt"][1] == "feed-forward":
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape(
                (last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.SimpleRNN(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0]
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.Dense(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "recurrent" and components["dt"][1] == "recurrent":
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape(
                (last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.SimpleRNN(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
                return_sequences=True
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.SimpleRNN(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "recurrent" and components["dt"][1] == "GRU":
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape(
                (last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.SimpleRNN(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
                return_sequences=True
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.GRU(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "recurrent" and components["dt"][1] == "LSTM":
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape(
                (last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.SimpleRNN(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
                return_sequences=True
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.LSTM(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "GRU" and components["dt"][1] == "feed-forward":
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape(
                (last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.GRU(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.Dense(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "GRU" and components["dt"][1] == "recurrent":
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape(
                (last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.GRU(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
                return_sequences=True
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.SimpleRNN(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "GRU" and components["dt"][1] == "GRU":
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape(
                (last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.GRU(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
                return_sequences=True
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.GRU(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "GRU" and components["dt"][1] == "LSTM":
            last_shape = new_model.layers[-1].output_shape
            new_model.add(tf.keras.layers.Reshape(
                (last_shape[1] * last_shape[2], last_shape[3]), 
                input_shape=last_shape
            ))
            new_model.add(layers.GRU(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
                return_sequences=True
            ))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.LSTM(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "LSTM" and components["dt"][1] == "feed-forward":
            new_model.add(layers.TimeDistributed(layers.LSTM(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
            )))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.Flatten())
            new_model.add(layers.Dense(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "LSTM" and components["dt"][1] == "recurrent":
            new_model.add(layers.TimeDistributed(layers.LSTM(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
            )))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.SimpleRNN(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "LSTM" and components["dt"][1] == "GRU":
            new_model.add(layers.TimeDistributed(layers.LSTM(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
            )))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.GRU(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))

        if components["dt"][0] == "LSTM" and components["dt"][1] == "LSTM":
            new_model.add(layers.TimeDistributed(layers.LSTM(
                components["dn"][0],
                kernel_regularizer=RGL_DCT[components["dr"][0]], 
                activation=components["da"][0],
            )))
            if components["dd"][0] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.LSTM(
                components["dn"][1],
                kernel_regularizer=RGL_DCT[components["dr"][1]], 
                activation=components["da"][1]
            ))
            if components["dd"][1] == 0.5:
                new_model.add(layers.Dropout(0.5))
            new_model.add(layers.Flatten())

    new_model.add(layers.Dense(10))

    return new_model

def build_model(components):
        model = models.Sequential()
        model.add(layers.Conv2D(
            components["ck"][0],
            (components["cs"][0], components["cs"][0]),
            activation=components["ca"][0],
            input_shape=(28, 28, 1)
        ))
        model.add(layers.MaxPooling2D((components["cp"][0], components["cp"][0])))

        for i in range(1, components["nc"]):
            model.add(layers.Conv2D(
                components["ck"][i],
                (components["cs"][i], components["cs"][i]),
                activation=components["ca"][i],
            ))
            model.add(layers.MaxPooling2D((components["cp"][i], components["cp"][i])))

        model = add_dense_layers(components, model)

        return model

def get_optimizers(function, lr):
        learning_rate = lr
        if function == 0:
            opt = optimizers.SGD(learning_rate=learning_rate)
        elif function == 1:
            opt = optimizers.SGD(learning_rate=learning_rate, momentum=0.1)
        elif function == 2:
            opt = optimizers.SGD(learning_rate=learning_rate, nesterov=True)
        elif function == 3:
            opt = optimizers.Adagrad(learning_rate=learning_rate)
        elif function == 4:
            opt = optimizers.Adamax(learning_rate=learning_rate)
        elif function == 5:
            opt = optimizers.Adam(learning_rate=learning_rate)
        elif function == 6:
            opt = optimizers.Adadelta(learning_rate=learning_rate)
        elif function == 7:
            opt = optimizers.RMSprop(learning_rate=learning_rate)

        return opt

def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
    x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
    x_train, x_test = x_train.astype("float32"), x_test.astype("float32")
    x_train, x_test = x_train / 255., x_test / 255.

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

In [ ]:
x_train, y_train, x_test, y_test = load_data()
components = get_components(GENE)
model = build_model(components)
opt = get_optimizers(components["f"], components["n"])
model.compile(
    optimizer=opt,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.summary()
batch_size = components["b"]
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), batch_size=batch_size)
_, test_acc = model.evaluate(x_test,  y_test)